In [1]:
from FundData import *
from helper import Helper
import pprint, json
import fitz

#dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'
#fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"
fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'

mutual_fund = Helper.get_fund_paths(fund_path)

In [2]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

        # Sort extracted blocks
        sorted_blocks = sorted(extracted_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))

        final_list.append({
            "pgn": pgn,
            "blocks": sorted_blocks
        })

    doc.close()

    return final_list


In [3]:
def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        for block in blocks.get('blocks',[]):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if span['color'] == -1361884:
                        spans.append({
                            "text": span.get("text"),
                            "color": span.get("color"),
                            "size": span.get("size"),
                            "font": span.get("font")
                        })
        final_span.append(spans)
    return final_span

In [58]:
def fund_report(data:dict, pattern:str):
    
    sizes = set()
    fonts = set()
    colors = set()
    
    for pgn,blocks in enumerate(data):
        for block in blocks.get('blocks',[]):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if span['font'] == 'Ubuntu-Bold' and span['color'] == -14475488:
                        print(pgn)
                        print(span['text'], span['size'],span['color'])
    

In [2]:
sample_path  = mutual_fund['Tata Mutual Fund']
data = Helper.get_all_pdf_data(sample_path)
len(data)

113

In [4]:
data[45]

{'pgn': 45,
 'blocks': [{'number': 55,
   'type': 0,
   'bbox': (22.339399337768555,
    23.631122589111328,
    276.80340576171875,
    42.81512451171875),
   'lines': [{'spans': [{'size': 16.0,
       'flags': 0,
       'font': 'Swiss721BT-BlackCondense',
       'color': -1,
       'ascender': 0.9629999995231628,
       'descender': -0.23600000143051147,
       'text': 'Tata Nifty Capital Markets Index Fund',
       'origin': (22.339399337768555, 39.03912353515625),
       'bbox': (22.339399337768555,
        23.631122589111328,
        276.80340576171875,
        42.81512451171875)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (22.339399337768555,
      23.631122589111328,
      276.80340576171875,
      42.81512451171875)}]},
  {'number': 56,
   'type': 0,
   'bbox': (22.339399337768555,
    42.77467346191406,
    210.9254150390625,
    49.96867370605469),
   'lines': [{'spans': [{'size': 6.0,
       'flags': 0,
       'font': 'Swiss721BT-RomanCondense',
       'color': 

In [42]:
# Lines to be drawn (start and end points)
lines = [
    ((285, 0), (285, 1000)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0,5,225,812)]

sample_path  = mutual_fund['Groww Mutual Fund']
pages = [i for i in range(1,70,1)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dry_path)
data = Helper.get_all_pdf_data(sample_path)

Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf
